In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession
    .builder
    .appName("spark_query")
    .config("spark.jars", "/home/long.vk@citigo.id/Downloads/mysql-connector-j-8.0.33.jar")
    .getOrCreate()
)

24/08/05 17:03:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/05 17:03:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
url = "jdbc:mysql://localhost:3306"

properties = {
    "user": "admin",
    "password": "password",
    "driver": "com.mysql.jdbc.Driver"
}

In [6]:
df_acc_creation = (
    spark
    .read
    .format("jdbc")
    .option("url", f"{url}/spark_prj_1_sample_dw")
    .option("driver", properties["driver"])
    .option("dbtable", "account_creation_fact")
    .option("user", properties["user"])
    .option("password", properties["password"])
    .load()
)

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


In [7]:
df_acc_creation.show(5)

+------+-------+------------+
|acc_id|user_id|created_date|
+------+-------+------------+
|     1|      1|  2023-06-20|
|     2|      1|  2022-02-01|
|     3|      2|  2018-08-01|
|     4|      2|  2018-11-09|
|     5|      2|  2024-03-14|
+------+-------+------------+
only showing top 5 rows



In [8]:
df_acc_creation.count()

2000992

In [10]:
df_acc_creation.createOrReplaceTempView("df")

In [13]:
spark.sql("""
    SELECT count(*) AS number_of_account
    FROM df
    WHERE df.created_date > '2024-01-01'
""").show()

+-----------------+
|number_of_account|
+-----------------+
|           178741|
+-----------------+



In [14]:
(
    spark
    .read
    .format("jdbc")
    .option("url", f"{url}/spark_prj_1_sample_dw")
    .option("driver", properties["driver"])
    .option("dbtable", "account_dim")
    .option("user", properties["user"])
    .option("password", properties["password"])
    .load()
).createOrReplaceTempView("account_dim")

In [15]:
(
    spark
    .read
    .format("jdbc")
    .option("url", f"{url}/spark_prj_1_sample_dw")
    .option("driver", properties["driver"])
    .option("dbtable", "user_dim")
    .option("user", properties["user"])
    .option("password", properties["password"])
    .load()
).createOrReplaceTempView("user_dim")

In [25]:
spark.sql("""
    SELECT 
            ud.user_id,
            CONCAT(first_name, ' ', last_name) as full_name,
            number_of_account
    FROM user_dim as ud
    JOIN (
        SELECT ud.user_id,
               COALESCE(COUNT(df.user_id), 0) AS number_of_account
        FROM user_dim ud
        LEFT JOIN df ON ud.user_id = df.user_id
        GROUP BY ud.user_id
        ORDER BY ud.user_id
    ) as tmp
    ON ud.user_id = tmp.user_id
    ORDER BY ud.user_id
""").show()

+-------+----------------+-----------------+
|user_id|       full_name|number_of_account|
+-------+----------------+-----------------+
|      1|     Paul Wright|                2|
|      2|      Ruth James|                4|
|      3| Fernando Pineda|                3|
|      4|Christopher Webb|                0|
|      5|    Joseph Scott|                2|
|      6|    Candice Snow|                0|
|      7|   David Hawkins|                3|
|      8| Jennifer Harper|                4|
|      9|     Sonya Blake|                3|
|     10|      Mary Garza|                3|
|     11|   Jessica Brown|                3|
|     12|    James Malone|                2|
|     13|  Jessica Gordon|                0|
|     14|      David Lane|                2|
|     15| Tiffany Anthony|                4|
|     16|    Terri Mathis|                3|
|     17| William Watkins|                1|
|     18|    Bruce Taylor|                2|
|     19|    Tamara Ortiz|                3|
|     20| 

In [35]:
spark.sql("""
    SELECT 
        number_of_account, AVG(age)
    FROM user_dim ud
    JOIN (
        SELECT df.user_id,
                count(df.user_id) as number_of_account
        FROM df
        GROUP BY df.user_id
    ) as tmp
    ON ud.user_id = tmp.user_id
    GROUP BY number_of_account
""").show()

+-----------------+------------------+
|number_of_account|          avg(age)|
+-----------------+------------------+
|                1|42.523720018796055|
|                3|42.506787285185666|
|                2|42.525144737766276|
|                4|42.536154418706886|
+-----------------+------------------+



In [40]:
spark.sql("""
    SELECT 
        AVG(age)
    FROM user_dim ud
    JOIN (
        SELECT df.user_id,
                count(df.user_id) as number_of_account
        FROM df
        GROUP BY df.user_id
        HAVING number_of_account > 2
    ) as tmp
    ON ud.user_id = tmp.user_id
    WHERE age > 0
""").show()

+-----------------+
|         avg(age)|
+-----------------+
|42.52145842951798|
+-----------------+

